In [31]:
import os
import pandas as pd
from shapely import wkb
from shapely.geometry import MultiPolygon
import plotly.graph_objects as go
import plotly.io as pio
import squidpy as sq

In [33]:
# ==============================================
# Get folder of current Python file
output_path_name = os.getcwd()

# Files
counts_file_name = 'cell_by_gene_3D_embryo4.csv'
meta_file_name = 'cell_metadata_3D_embryo4.csv'
parquetpath = os.path.join(output_path_name, 'cellpose_micron_space.parquet')

# ==============================================
# Load spatial transcriptomics data
sliceX = sq.read.vizgen(
    path=output_path_name,
    counts_file=counts_file_name,
    meta_file=meta_file_name,
    transformation_file="micron_to_mosaic_pixel_transform.csv",
)

# ==============================================
# Define cluster colors (do this before using it)
layer_to_color_map = {
    'cluster_0': '#00aa00', 
    'cluster_1': '#55007f', 
    'cluster_2': '#ffaa00', 
    'cluster_3': '#00aaff', 
    'cluster_4': '#ff557f',
    'cluster_5': '#e6e6e6'
}

# Assign colors
sliceX.obs['cluster'] = sliceX.obs['cluster'].astype('category')
sliceX.uns['cluster_colors'] = sliceX.obs['cluster'].map(lambda c: layer_to_color_map[str(c)]).tolist()

# ==============================================
# Load parquet geometry data
parquetdata = pd.read_parquet(parquetpath)
print('Parquet data Z-levels:', set(parquetdata['ZLevel'].tolist()))

# Filter for specific Z index
parquetdata_zindex = parquetdata[parquetdata['ZIndex'] == 1]

# Load metadata & filter
selected_cell_meta = pd.read_csv(meta_file_name)
parquetdata_sel = parquetdata_zindex[parquetdata_zindex['EntityID'].isin(selected_cell_meta['EntityID'].tolist())]

# ==============================================
# Convert WKB to shapely geometry
parquetdata_sel['Geometry_1'] = [wkb.loads(g) for g in parquetdata_sel['Geometry']]

# Assign z-value & colors
parquetdata_sel['z_value'] = 7  # fixed height
parquetdata_sel['cluster'] = sliceX.obs['cluster'].tolist()
parquetdata_sel['color'] = parquetdata_sel['cluster'].map(layer_to_color_map)
parquetdata_sel



Parquet data Z-levels: {1.5, 3.0, 4.5, 6.0, 7.5, 9.0, 10.5}


/tmp/ipykernel_3988096/2076931396.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3988096/2076931396.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3988096/2076931396.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ip

,ID,EntityID,ZIndex,Geometry,ParentType,ParentID,Type,ZLevel,Name,Geometry_1,z_value,cluster,color
57672,57672,3066494500068100153,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6241.622750609909 6901.6434976...,7,cluster_5,#e6e6e6
2921,2921,3066494500068100157,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6251.110119455814 6904.3780521...,7,cluster_5,#e6e6e6
2922,2922,3066494500068100161,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6293.129500789524 6913.1451272...,7,cluster_5,#e6e6e6
2923,2923,3066494500068100162,1,b'\x01\x06\x00\x00\x00\x03\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6263.560571564267 6912.5413267...,7,cluster_5,#e6e6e6
2924,2924,3066494500068100163,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6276.185262319955 6913.9011071...,7,cluster_5,#e6e6e6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77253,77253,3066494500093200043,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6408.9628875092985 7490.915736...,7,cluster_5,#e6e6e6
77254,77254,3066494500093200045,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6404.567709816438 7500.2433639...,7,cluster_5,#e6e6e6
77256,77256,3066494500093200050,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6403.652135173684 7505.7514122...,7,cluster_5,#e6e6e6
4207,4207,3066494500093200055,1,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,None,None,cell,3.0,None,MULTIPOLYGON (((6402.946406959558 7511.4088858...,7,cluster_5,#e6e6e6


In [34]:

# ==============================================
# Prepare Plotly figure
fig = go.Figure()

for idx, row in parquetdata_sel.iterrows():
    geom = row['Geometry_1']
    color = row['color']

    if isinstance(geom, MultiPolygon):
        polygons = geom.geoms
    else:
        polygons = [geom]  # Handle single Polygon

    for poly in polygons:
        x, y = poly.exterior.xy
        x_list, y_list = list(x), list(y)

        # Close polygon loop
        if (x_list[0], y_list[0]) != (x_list[-1], y_list[-1]):
            x_list.append(x_list[0])
            y_list.append(y_list[0])

        fig.add_trace(go.Scatter(
            x=x_list,
            y=y_list,
            mode='lines',
            fill='toself',
            fillcolor=color,
            line=dict(color='black', width=0.5),
            name=f'Polygon {idx + 1}',
            showlegend=False
        ))

# Layout
fig.update_layout(
    title="2D Multipolygon Plot",
    xaxis_title="X Axis",
    yaxis_title="Y Axis",
    yaxis=dict(scaleanchor="x", scaleratio=1),
    template="simple_white"
)

# Save to PDF
output_pdf = os.path.join(output_path_name, "multipolygon_plot.pdf")
pio.write_image(fig, output_pdf, format="pdf", width=800, height=600)

fig.show()